# TPS-Aug-2022

In [1]:
class Config:
    NB = '204'
    dataset_NB = '102'

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'id'
    target = 'failure'

## Import libralies

In [2]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)


color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

In [4]:
import random
import joblib
import itertools
from itertools import combinations

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score, roc_curve, auc
from lightgbm import LGBMClassifier, early_stopping

## Load and check data

In [5]:
df_train = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_train.pkl', compression='zip')
df_test = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_test.pkl', compression='zip')

submission = pd.read_csv(Config.raw_data_dir + 'sample_submission.csv', header=None)

df_train.shape

(26570, 335)

In [6]:
df_train.head()

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_6,attribute_1_material_7,attribute_1_material_8,mes_0x1,mes_0-1,mes_0x2,mes_0-2,mes_0x3,mes_0-3,mes_0x4,mes_0-4,mes_0x5,mes_0-5,mes_0x6,mes_0-6,mes_0x7,mes_0-7,mes_0x8,mes_0-8,mes_0x9,mes_0-9,mes_0x10,mes_0-10,mes_0x11,mes_0-11,mes_0x12,mes_0-12,mes_0x13,mes_0-13,mes_0x14,mes_0-14,mes_0x15,mes_0-15,mes_0x16,mes_0-16,mes_0x17,mes_0-17,mes_1x2,mes_1-2,mes_1x3,mes_1-3,mes_1x4,mes_1-4,mes_1x5,mes_1-5,mes_1x6,mes_1-6,mes_1x7,mes_1-7,mes_1x8,mes_1-8,mes_1x9,mes_1-9,mes_1x10,mes_1-10,mes_1x11,mes_1-11,mes_1x12,mes_1-12,mes_1x13,mes_1-13,mes_1x14,mes_1-14,mes_1x15,mes_1-15,mes_1x16,mes_1-16,mes_1x17,mes_1-17,mes_2x3,mes_2-3,mes_2x4,mes_2-4,mes_2x5,mes_2-5,mes_2x6,mes_2-6,mes_2x7,mes_2-7,mes_2x8,mes_2-8,mes_2x9,mes_2-9,mes_2x10,mes_2-10,mes_2x11,mes_2-11,mes_2x12,mes_2-12,mes_2x13,mes_2-13,mes_2x14,mes_2-14,mes_2x15,mes_2-15,mes_2x16,mes_2-16,mes_2x17,mes_2-17,mes_3x4,mes_3-4,mes_3x5,mes_3-5,mes_3x6,mes_3-6,mes_3x7,mes_3-7,mes_3x8,mes_3-8,mes_3x9,mes_3-9,mes_3x10,mes_3-10,mes_3x11,mes_3-11,mes_3x12,mes_3-12,mes_3x13,mes_3-13,mes_3x14,mes_3-14,mes_3x15,mes_3-15,mes_3x16,mes_3-16,mes_3x17,mes_3-17,mes_4x5,mes_4-5,mes_4x6,mes_4-6,mes_4x7,mes_4-7,mes_4x8,mes_4-8,mes_4x9,mes_4-9,mes_4x10,mes_4-10,mes_4x11,mes_4-11,mes_4x12,mes_4-12,mes_4x13,mes_4-13,mes_4x14,mes_4-14,mes_4x15,mes_4-15,mes_4x16,mes_4-16,mes_4x17,mes_4-17,mes_5x6,mes_5-6,mes_5x7,mes_5-7,mes_5x8,mes_5-8,mes_5x9,mes_5-9,mes_5x10,mes_5-10,mes_5x11,mes_5-11,mes_5x12,mes_5-12,mes_5x13,mes_5-13,mes_5x14,mes_5-14,mes_5x15,mes_5-15,mes_5x16,mes_5-16,mes_5x17,mes_5-17,mes_6x7,mes_6-7,mes_6x8,mes_6-8,mes_6x9,mes_6-9,mes_6x10,mes_6-10,mes_6x11,mes_6-11,mes_6x12,mes_6-12,mes_6x13,mes_6-13,mes_6x14,mes_6-14,mes_6x15,mes_6-15,mes_6x16,mes_6-16,mes_6x17,mes_6-17,mes_7x8,mes_7-8,mes_7x9,mes_7-9,mes_7x10,mes_7-10,mes_7x11,mes_7-11,mes_7x12,mes_7-12,mes_7x13,mes_7-13,mes_7x14,mes_7-14,mes_7x15,mes_7-15,mes_7x16,mes_7-16,mes_7x17,mes_7-17,mes_8x9,mes_8-9,mes_8x10,mes_8-10,mes_8x11,mes_8-11,mes_8x12,mes_8-12,mes_8x13,mes_8-13,mes_8x14,mes_8-14,mes_8x15,mes_8-15,mes_8x16,mes_8-16,mes_8x17,mes_8-17,mes_9x10,mes_9-10,mes_9x11,mes_9-11,mes_9x12,mes_9-12,mes_9x13,mes_9-13,mes_9x14,mes_9-14,mes_9x15,mes_9-15,mes_9x16,mes_9-16,mes_9x17,mes_9-17,mes_10x11,mes_10-11,mes_10x12,mes_10-12,mes_10x13,mes_10-13,mes_10x14,mes_10-14,mes_10x15,mes_10-15,mes_10x16,mes_10-16,mes_10x17,mes_10-17,mes_11x12,mes_11-12,mes_11x13,mes_11-13,mes_11x14,mes_11-14,mes_11x15,mes_11-15,mes_11x16,mes_11-16,mes_11x17,mes_11-17,mes_12x13,mes_12-13,mes_12x14,mes_12-14,mes_12x15,mes_12-15,mes_12x16,mes_12-16,mes_12x17,mes_12-17,mes_13x14,mes_13-14,mes_13x15,mes_13-15,mes_13x16,mes_13-16,mes_13x17,mes_13-17,mes_14x15,mes_14-15,mes_14x16,mes_14-16,mes_14x17,mes_14-17,mes_15x16,mes_15-16,mes_15x17,mes_15-17,mes_16x17,mes_16-17
0,0,80.10,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,16.0265,13.034,14.684,764.100,0.0,0,1,0,0,0,1,56,1,28,3,126.280,11.040,87.626,5.518,110.236,8.748,135.044,12.292,82.173,4.739,141.085,13.155,74.704,3.672,111.013,8.859,123.158,10.594,106.351,8.193,105.203,8.029,112.1855,9.0265,91.238,6.034,102.788,7.684,5348.700,757.100,32,4,144.320,10.040,100.144,4.518,125.984,7.748,154.336,11.292,93.912,3.739,161.240,12.155,85.376,2.672,126.872,7.859,140.752,9.594,121.544,7.193,120.232,7.029,128.212,8.0265,104.272,5.034,117.472,6.684,6112.800,756.100,72.160,14.040,50.072,8.518,62.992,11.748,77.168,15.292,46.956,7.739,80.620,16.155,42.688,6.672,63.436,11.859,70.376,13.594,60.772,11.193,60.116,11.029,64.106,12.0265,52.136,9.034,58.736,10.684,3056.400,760.100,225.824720,5.522,284.093920,2.292,348.02

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Columns: 335 entries, id to mes_16-17
dtypes: float64(317), int64(12), uint8(6)
memory usage: 66.8 MB


In [8]:
def seed_everything(seed):

    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [9]:
# Get feature list
features = [col for col in df_train.columns if col not in [Config.row_id, Config.target]]

'''
features = [col for col in df_train.columns if 'mes_' not in col]
features.remove(Config.row_id)
features.remove(Config.target)
'''
features

['loading',
 'attribute_2',
 'attribute_3',
 'measurement_0',
 'measurement_1',
 'measurement_2',
 'measurement_3',
 'measurement_4',
 'measurement_5',
 'measurement_6',
 'measurement_7',
 'measurement_8',
 'measurement_9',
 'measurement_10',
 'measurement_11',
 'measurement_12',
 'measurement_13',
 'measurement_14',
 'measurement_15',
 'measurement_16',
 'measurement_17',
 'attribute_0_material_5',
 'attribute_0_material_7',
 'attribute_1_material_5',
 'attribute_1_material_6',
 'attribute_1_material_7',
 'attribute_1_material_8',
 'mes_0x1',
 'mes_0-1',
 'mes_0x2',
 'mes_0-2',
 'mes_0x3',
 'mes_0-3',
 'mes_0x4',
 'mes_0-4',
 'mes_0x5',
 'mes_0-5',
 'mes_0x6',
 'mes_0-6',
 'mes_0x7',
 'mes_0-7',
 'mes_0x8',
 'mes_0-8',
 'mes_0x9',
 'mes_0-9',
 'mes_0x10',
 'mes_0-10',
 'mes_0x11',
 'mes_0-11',
 'mes_0x12',
 'mes_0-12',
 'mes_0x13',
 'mes_0-13',
 'mes_0x14',
 'mes_0-14',
 'mes_0x15',
 'mes_0-15',
 'mes_0x16',
 'mes_0-16',
 'mes_0x17',
 'mes_0-17',
 'mes_1x2',
 'mes_1-2',
 'mes_1x3',
 '

In [10]:
# 約40分

# Create a numpy array to store test predictions
test_predictions = np.zeros(len(df_test))

# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(df_train))

feature_importance_df = pd.DataFrame(index=features)
y_valids, val_preds =[],[]
amex_scores = []

kfold = StratifiedKFold(n_splits = Config.n_folds, shuffle = True, random_state = Config.random_seed)

for fold, (train_idx, valid_idx) in enumerate(kfold.split(df_train, df_train[Config.target])):

    print(' ')
    print('-'*50)
    print(f'Training fold {fold+1} with {len(features)} features...')

    x_train, x_val = df_train[features].iloc[train_idx], df_train[features].iloc[valid_idx]
    y_train, y_val = df_train[Config.target].iloc[train_idx], df_train[Config.target].iloc[valid_idx]

    model = LogisticRegression()
    model.fit(x_train, y_train)

    print(f'================================== training {fold+1} fin. ==================================')

    # Predict validation data
    print(f'================================== validation-data predicting ... ==================================')
    val_pred = model.predict_proba(x_val)[:, 1]
    oof_predictions[valid_idx] = val_pred

    # Predict test data
    print(f'================================== test-data predicting ... ==================================')
    test_pred = model.predict_proba(df_test[features])[:, 1]
    test_predictions += test_pred / Config.n_folds

    # save results
    y_valids.append(y_val)
    val_preds.append(val_pred)
    feature_importance_df["Importance_Fold"+str(fold+1)]=model.coef_.ravel()

    # Compute fold metric
    val_pred = pd.DataFrame(data={'prediction': val_pred})
    y_val = pd.DataFrame(data={'target': y_val.reset_index(drop=True)})
    auc_score = roc_auc_score(y_val, val_pred)

    print(f'Fold {fold+1} CV result')
    print(f' ROC metric : {auc_score}')

    del x_train, x_val, y_train, y_val
    _ = gc.collect()

# Compute out of folds metric
oof_predictions = pd.DataFrame(data={'prediction': oof_predictions})
y_true = pd.DataFrame(data={Config.target: df_train[Config.target]})

print(' ')
print('-'*50)
print(f'TOTAL AUC socre : {roc_auc_score(df_train[Config.target], oof_predictions["prediction"])}')
print('-'*50)

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({Config.row_id: df_train[Config.row_id], Config.target: df_train[Config.target], 'prediction': oof_predictions['prediction']})

# Create a dataframe to store test prediction
test_df = pd.DataFrame({Config.row_id: df_test[Config.row_id], Config.target: test_predictions})

 
--------------------------------------------------
Training fold 1 with 333 features...
================================== training 1 fin. ==================================
================================== validation-data predicting ... ==================================
================================== test-data predicting ... ==================================
Fold 1 CV result
 ROC metric : 0.5346508312935925
 
--------------------------------------------------
Training fold 2 with 333 features...
================================== training 2 fin. ==================================
================================== validation-data predicting ... ==================================
================================== test-data predicting ... ==================================
Fold 2 CV result
 ROC metric : 0.5843785427841419
 
--------------------------------------------------
Training fold 3 with 333 features...
================================== training 3 fin. ===============

In [ ]:
# Save results
# oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
# test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)

In [11]:
oof_df.head()

,id,failure,prediction
0,0,0.0,0.238706
1,1,0.0,0.217764
2,2,0.0,0.195925
3,3,0.0,0.205729
4,4,0.0,0.265910


In [12]:
def plot_roc(y_val, y_prob):
    #colors=px.colors.qualitative.Prism
    fig = go.Figure(layout=plotly_template['layout'])
    fig.add_trace(go.Scatter(x=np.linspace(0,1,11), y=np.linspace(0,1,11), name='Random Chance', mode='lines', showlegend=False, line=dict(color="Black", width=1, dash="dot")))

    for i in range(len(y_val)):
        y=y_val[i]
        prob=y_prob[i]
        fpr, tpr, _ = roc_curve(y, prob)
        roc_auc = auc(fpr,tpr)
        fig.add_trace(go.Scatter(x=fpr, y=tpr, line=dict(color=color_palette['Cat5'][i], width=3),
                                 hovertemplate = 'True positive rate = %{y:.3f}<br>False positive rate = %{x:.3f}',
                                 name='Fold {}: AUC = {:.3f}'.format(i+1, roc_auc)))

    fig.update_layout(template=plotly_template, title="Cross-Validation ROC Curves",
                      hovermode="x unified", width=700, height=600,
                      xaxis_title='False Positive Rate (1 - Specificity)',
                      yaxis_title='True Positive Rate (Sensitivity)',
                      legend=dict(orientation='v', y=.07, x=1, xanchor="right",
                                  bordercolor="black", borderwidth=.5))
    fig.show()

plot_roc(y_valids, val_preds)

In [13]:
top = 50

feature_importance_df['avg'] = feature_importance_df.mean(axis=1)
feature_importance_top = feature_importance_df.avg.nlargest(top).sort_values(ascending=True)

pal=sns.color_palette("YlGnBu", 65).as_hex()
fig=go.Figure()
for i in range(len(feature_importance_top.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feature_importance_top[i],
                       line_color=pal[::-1][i],opacity=0.8,line_width=4))

fig.add_trace(go.Scatter(x=feature_importance_top, y=feature_importance_top.index, mode='markers',
                         marker_color=pal[::-1], marker_size=8,
                         hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))

fig.update_layout(template=plotly_template,title=f'LGBM Feature Importance<br>Top {top}',
                  margin=dict(l=150,t=80),
                  xaxis=dict(title='Importance', zeroline=False),
                  yaxis_showgrid=False, height=1000, width=800)
fig.show()

In [14]:
# test_df = pd.DataFrame({Config.row_id: test[Config.row_id], 'prediction': test_predictions})

df = pd.DataFrame(data={'Target':test_df[Config.target].apply(lambda x: 1 if x>0.25 else 0)})
df = df.Target.value_counts(normalize=True)
df.rename(index={1:'Positive', 0:'Negative'}, inplace=True)

#pal, color=['#016CC9','#DEB078'], ['#8DBAE2','#EDD3B3']
fig=go.Figure()

fig.add_trace(go.Pie(labels=df.index, values=df*100, hole=.45,
                     showlegend=True,sort=False,
                     marker=dict(colors=color_palette['Bin'],line=dict(color=pal,width=2.5)),
                     hovertemplate = "%{label}: %{value:.2f}%<extra></extra>"))

fig.update_layout(template=plotly_template, title='Predicted Target Distribution',
                  legend=dict(traceorder='reversed',y=1.05,x=0),
                  uniformtext_minsize=15, uniformtext_mode='hide',width=700)
fig.show()

In [15]:
test_df

,id,failure
26570,26570,0.234440
26571,26571,0.160061
26572,26572,0.210795
26573,26573,0.211767
26574,26574,0.274325
...,...,...
47340,47340,0.194347
47341,47341,0.193220
47342,47342,0.166599
47343,47343,0.201054


In [16]:
test_df[Config.target].describe()

count    20775.000000
mean         0.212361
std          0.034457
min          0.112081
25%          0.188006
50%          0.207530
75%          0.231325
max          0.451861
Name: failure, dtype: float64

In [17]:
test_df.to_csv(Config.submission_dir + f'nb{Config.NB}.csv', index=False)

## 検証メモ